Imports

In [1]:
from src.vitaldb.fetchingstrategy.VitalFileApi import VitalFileApi
from src.vitaldb.casegenerator import VitalFileOptions
from src.vitaldb.casesplit import split_generator
from src.metrics.standardeviation import StandardDeviation, AbsoluteError
import src.preprocessing.transforms as transforms
import src.preprocessing.filters as filters

import tensorflow as tf

from tensorflow import keras
from keras import layers
from keras.metrics import MeanAbsoluteError
import pandas as pd

Create the datasets

In [2]:
frequency = 500
samples = range(1, 100)
train_split = 0.7
validate_split = 0.15
validate_test = 0.15
batching = False

options = VitalFileOptions(
    ['SNUADC/ART'],
    1/frequency
)

train_generator, val_generator, test_generator = split_generator(options, VitalFileApi(), samples, [0.7, 0.15, 0.15])

dataset_train = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 1), dtype=tf.float64)
    )
).take(1)

dataset_val = tf.data.Dataset.from_generator(
    lambda: val_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, 1), dtype=tf.float64)
    )
).take(1)

2022-06-29 16:33:05.571804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Preprocess

In [3]:
def abp_low_pass_graph_adapter(x, frequency):
    return tf.numpy_function(transforms.abp_low_pass, [x, frequency], tf.float64)

def extract_clean_windows_graph_adapter(x, frequency: int, window_size: int, step_size: int):
    return tf.numpy_function(transforms.extract_clean_windows, [x, frequency, window_size, step_size], tf.float64)

def preprocess_dataset(dataset: tf.data.Dataset):
    dataset = dataset.filter(filters.has_data)
    dataset = dataset.map(transforms.remove_nan)
    dataset = dataset.map(lambda x: abp_low_pass_graph_adapter(x, frequency))
    dataset = dataset.map(lambda x: extract_clean_windows_graph_adapter(x, frequency, 8, 2))
    dataset= dataset.flat_map(transforms.to_tensor)
    dataset = dataset.filter(lambda x: filters.pressure_out_of_bounds(x, 30, 230))
    dataset = dataset.map(transforms.extract_sbp_dbp_from_abp_window)
    dataset = dataset.map(transforms.scale_array)

    if batching:
        dataset = dataset.map(lambda d, l: (tf.reshape(d, shape=(4000, 1)), l))
        dataset = dataset.batch(20).prefetch(2)
    else:
        dataset = dataset.map(lambda d, l: (tf.reshape(d, shape=(1, 4000)), l))

    return dataset

dataset_train = preprocess_dataset(dataset_train)
dataset_val = preprocess_dataset(dataset_val)

Define the model

In [4]:
input_shape = (4000, 1)
inputs = keras.Input(shape=input_shape)
x = layers.Conv1D(64, 15, activation='relu', input_shape=(4000, 1))(inputs)
x = layers.BatchNormalization()(x)
x = layers.MaxPooling1D(4)(x)
x = layers.Dropout(0.1)(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.LSTM(64)(x)
outputs = layers.Dense(2)(x)

model = keras.Model(inputs, outputs, name='CNN_LSTM')
model.summary()
model.compile(optimizer='Adam', loss=keras.losses.MeanAbsoluteError(),
    metrics = [
      MeanAbsoluteError(),
      StandardDeviation(AbsoluteError())
    ]
)

Model: "CNN_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 4000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 3986, 64)          1024      
                                                                 
 batch_normalization (BatchN  (None, 3986, 64)         256       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 996, 64)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 996, 64)           0         
                                                                 
 lstm (LSTM)                 (None, 996, 64)           330

train the model

In [5]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs", histogram_freq = 1, profile_batch = '500,520')
model.fit(dataset_train, epochs=10, callbacks=[tensorboard_callback], validation_data=dataset_val)

Epoch 1/10


/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/scipy/interpolate/_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/heartpy/analysis.py:778: RuntimeWarning: invalid value encountered in double_scalars
  measures['sd1/sd2'] = sd1 / sd2
/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-py3.10/lib/python3.10/site-packages/numpy/ma/core.py:5288: RuntimeWarning: Mean of empty slice.
  result = super().mean(axis=axis, dtype=dtype, **kwargs)[()]
/Users/francois/Library/Caches/pypoetry/virtualenvs/bpbaseline-d4rfio-H-p

1599/1599 [==============================] - 488s 293ms/step - loss: 48.6980 - mean_absolute_error: 48.6980 - standard_deviation: 50.0455
Epoch 2/10
1599/1599 [==============================] - 0s 210us/step - loss: 48.6980 - mean_absolute_error: 48.6980 - standard_deviation: 50.0455


Evaluate the model

In [7]:
%load_ext  tensorboard
%tensorboard --logdir=logs

ModuleNotFoundError: No module named ' tensorboard'

In [ ]:
model.save('models')